<a href="https://colab.research.google.com/github/drhoffma/HarperIDEAShopCVCC/blob/main/lessons/05_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://github.com/drhoffma/HarperIDEAShopCVCC/blob/main/colab_template/header_image.png?raw=true)

# Welcome to **Harper College's IDEAShop** curriculum!

**Course Title:** *Computer Vision, Cameras, & Code*

**Quick Links:** [Homepage](https://tinyurl.com/harpercv-home) | [Syllabus](https://tinyurl.com/harpercv-syllabus) | [Blackboard](#) | [Instructor](https://davidrhoffman.net)

***Terms of use:*** *This course is public domain. If you'd like to use content from this course, please request consent by [contacting Harper College MakerSpace](https://www.harpercollege.edu/maker/contact/index.php).*

---

# Deep Learning

* Image Classification - is the entire image representative of a "dog"?
* Object Detection - where is the "dog" in the image?
* Segmentation - highlight all the pixels where the "dog" is in the image

(replace "dog" with your favorite class of data)

## Training

In [ ]:
# imports
import cv2
# import keras, tensorflow, or pytorch

# load a dataset

# sanitize the dataset

# build a model

# train the model on the data

# evaluate the model

# if it meets expectations, serialize (save) the model to disk

## Pre-trained Models (Perform Inference)

In [1]:
! wget https://github.com/drhoffma/HarperIDEAShopCVCC/raw/main/lessons/images/wine.jpg

--2021-07-28 16:34:42--  https://github.com/drhoffma/HarperIDEAShopCVCC/raw/main/lessons/images/wine.jpg
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/drhoffma/HarperIDEAShopCVCC/main/lessons/images/wine.jpg [following]
--2021-07-28 16:34:42--  https://raw.githubusercontent.com/drhoffma/HarperIDEAShopCVCC/main/lessons/images/wine.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32846 (32K) [image/jpeg]
Saving to: ‘wine.jpg.1’

wine.jpg.1          100%[===================>]  32.08K  --.-KB/s    in 0.001s  

2021-07-28 16:34:43 (32.6 MB/s) - ‘wine.jpg.1’ saved [32846/32846]



In [ ]:
# imports
import cv2
import numpy as np
from keras.preprocessing import image as image_utils
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.applications import VGG16

# load an image


# load a an ImageNet model

# perform inference

# display results